## VGG 网络结构

![](./img/VGG.png)

In [1]:
from torch import nn
from torch.utils.data import Dataset

class VGG_Block(nn.Module):
    def __init__(self,num_convs, in_channels, out_channels):
        super().__init__()
        
        layers=[]
        for i in range(num_convs):
            layers.append(nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1))
            
            layers.append(nn.ReLU())
            
            in_channels=out_channels
        
        layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
        
        self.net=nn.Sequential(*layers)
        
    def forward(self,X):
        return self.net(X)
    
    

conv_arch=((2, 64), (2, 128), (3, 256), (3, 512), (2, 128))
class VGG(nn.Module):
    def __init__(self,conv_arch,input_channels):
        super().__init__()
        
        layers=[]
        for (conv_num,out_channels) in conv_arch:
        
            layers.append(VGG_Block(conv_num,input_channels,out_channels))
            input_channels=out_channels
            
        
        self.net=nn.Sequential(
            *layers,
            nn.Flatten(),
            nn.Linear(6272,1000),nn.ReLU(),
            nn.Dropout(p=0.8),   
            nn.Linear(1000,10)
        )
        
        
    def forward(self,X):
        return self.net(X)
    

            
conv_arch_1=((2, 64), (2, 128), (2, 256), (2, 512), (1, 64))
class VGG_1(nn.Module):
    def __init__(self,conv_arch,input_channels):
        super().__init__()
        
        layers=[]
        for (conv_num,out_channels) in conv_arch:
        
            layers.append(VGG_Block(conv_num,input_channels,out_channels))
            input_channels=out_channels
            
        
        self.net=nn.Sequential(
            *layers,
            nn.Flatten(),
            nn.Linear(3136,392),nn.ReLU(),
            nn.Linear(392,10)
        )   
    def forward(self,X):
        return self.net(X)

conv_arch_2=((2, 64), (1, 128), (3, 256), (1, 256), (1, 64))
class VGG_2(nn.Module):
    def __init__(self,conv_arch,input_channels):
        super().__init__()
        
        layers=[]
        for (conv_num,out_channels) in conv_arch:
        
            layers.append(VGG_Block(conv_num,input_channels,out_channels))
            input_channels=out_channels
            
        
        self.net=nn.Sequential(
            *layers,
            nn.Flatten(),
            nn.Linear(3136,1000),nn.ReLU(),
            nn.Linear(1000,256),nn.ReLU(),
            nn.Linear(256,10)
        )   
        
    def forward(self,X):
        return self.net(X)

            

conv_arch_3=((3, 64), (1, 128), (1, 256), (1, 256), (1, 64))
class VGG_3(nn.Module):
    def __init__(self,conv_arch,input_channels):
        super().__init__()
        
        layers=[]
        for (conv_num,out_channels) in conv_arch:
        
            layers.append(VGG_Block(conv_num,input_channels,out_channels))
            input_channels=out_channels
            
        
        self.net=nn.Sequential(
            *layers,
            nn.Flatten(),
            nn.Linear(3136,1000),nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1000,256),nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256,10)
        )   
        
    def forward(self,X):
        return self.net(X)
        




In [2]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
aug=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([224,224]),
    transforms.ToTensor()]
)

class FashionMnist(Dataset):

    def __init__(self, img_path, label_path):
        super().__init__()
        self.load_data_from_path(img_path, label_path)

    def __getitem__(self, index):
        return self.imgs[index], self.labels[index]

    def __len__(self):
        return self.img_num

    def load_data_from_path(self, img_path, label_path):
        with open(img_path, 'rb') as f:
            s = f.read()

        self.img_num = int(s[4:8].hex(), 16)
        self.imgs = torch.FloatTensor(list(iter(s[16:])))
        # print(self.img_num,self.imgs)
        self.imgs = torch.reshape(self.imgs, (-1, 1, 28, 28))
        # print(self.imgs.shape)
        with open(label_path, 'rb') as f:
            s = f.read()
        self.labels = torch.tensor(list(iter(s[8:])))
        # print(self.labels.shape)



def convert_to_224(img,aug):
    return aug(img).numpy().tolist()


def collate_fn(batch):
    inputs, labels = [], []
    for X, y in batch:
        inputs.append(convert_to_224(X, aug))
        labels.append(y)
        
    
    return torch.Tensor(inputs), torch.tensor(labels)




In [3]:
train_img_path='D:/datasets/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte'
train_label_path='D:/datasets/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte'
test_img_path='D:/datasets/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte'
test_label_path='D:/datasets/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte'

In [4]:
train_data=FashionMnist(train_img_path,train_label_path)
test_data=FashionMnist(test_img_path,test_label_path)
train_iter=DataLoader(train_data,batch_size=16,shuffle=True,collate_fn=collate_fn)
test_iter=DataLoader(test_data,batch_size=16,shuffle=True,collate_fn=collate_fn)

In [5]:

def precise(test_iter,net,device):
    total=0
    correct=0
    with torch.no_grad():
        for X,y in test_iter:
            total+=len(X)
            X,y=X.to(device),y.to(device)
            y_hat=net(X)
            out_put=torch.argmax(y_hat,dim=-1)
            ans=(out_put==y)
            correct+=ans.sum().item()
        
    print(correct)
    print(f"accuracy :{correct/total*100:>3f}% ")

    
def train(data_iter,entroy_iter,net,optimizer,lr_scheduler,loss_fn,epochs,device,epoch_data_num):
    matrix_x,matrix_loss,entroy_loss,entroy_x=[0],[0],[],[]
    total_loss=0
    batchs=len(data_iter)
    for epoch in range(epochs):
        now_num=0
        for X,y in data_iter:
 
            now_num+=len(X)
    
            net.train()
            
            X,y=X.to(device),y.to(device)
            optimizer.zero_grad()
            y_hat=net(X)
            loss=loss_fn(y_hat,y)
            loss.sum().backward()

            optimizer.step()

            total_loss+=loss.item()

            matrix_x.append(matrix_x[-1]+1)
            matrix_loss.append(total_loss/(epoch*epoch_data_num+now_num))
            
            print(f"loss: {matrix_loss[-1]:>7f} now {matrix_x[-1]}/{batchs*epochs}",end='\r')
            
        
        lr_scheduler.step()
        with torch.no_grad():
            c_total_loss=0
            test_data_num=0
            for X,y in entroy_iter: 
                net.eval()
                test_data_num+=len(X)
                X,y=X.to(device),y.to(device)
                y_hat=net(X)
                loss=loss_fn(y_hat,y)
                
                c_total_loss+=loss.item()
            
            print(test_data_num)
            entroy_loss.append(c_total_loss/test_data_num)
            entroy_x.append((epoch+1)*batchs)
        print(f"cross entroy loss:{entroy_loss[-1]} now {epoch+1}/{epochs}")
        torch.save(net.state_dict(), f"Vgg_epoch{epoch+4}.bin")
        precise(test_iter,net,device)

  
    
    return net,matrix_x,matrix_loss,entroy_x,entroy_loss






In [6]:
device='cuda' if torch.cuda.is_available() else 'cpu'
net=VGG(conv_arch,1).to(device)
net.load_state_dict(torch.load('Vgg_epoch3.bin'))

<All keys matched successfully>

In [8]:
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
device='cuda' if torch.cuda.is_available() else 'cpu'


weights_path='./epoch1.bin'
optimizer=Adam(net.parameters(),lr=0.0000003)
lr_scheduler=LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(epoch+1))
loss_fn=nn.CrossEntropyLoss()
_,matrix_x,matrix_loss,entroy_x,entroy_loss=train(train_iter,test_iter,net,optimizer,lr_scheduler,loss_fn,1,device,len(train_data))


10000 0.013192 now 3750/3750
cross entroy loss:0.01425676906905137 now 1/1
9145
accuracy :91.450000% 
